In [ ]:
!pip3 install mtcnn

     |████████████████████████████████| 2.3MB 6.8MB/s 


In [ ]:
import keras
import tensorflow as tf
from keras.models import Model
from keras import backend as K
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Dense,Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import *
from sklearn import metrics
import math
import numpy as np
from sklearn.metrics import confusion_matrix
import math
import numpy as np
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from keras.models import load_model 
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from google.colab.patches import cv2_imshow
import cv2
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def face(fil):
  final_size=(160, 160)
  img = Image.open(fil)
  img = img.convert('RGB')
  arr = asarray(img)
  ret = MTCNN().detect_faces(arr)
  x1, y1, width, height = ret[0]['box']

  if int(height)==0 and int(width)==0:
    return arr

  x1, y1 = abs(x1), abs(y1)
  x2, y2 = x1 + width, y1 + height
  extracted_face = arr[y1:y2, x1:x2]
  img = Image.fromarray(extracted_face)
  img = img.resize(final_size)
  ans = asarray(img)
  return ans

In [ ]:
def make_X(dir):
  i=1
  X = list()
  path=glob.glob(dir)
  for src in path:
    X.append(face(src))
    print(i)
    i+=1
  return X

In [ ]:
def load(dir):
  X=list()
  y=list()
  for subdr in listdir(dir):
    path = dir + subdr +"/*jpg"
    f = make_X(path)
    label = [subdr for _ in range(len(f))]
    X.extend(f)
    y.extend(label)
  return asarray(X), asarray(y)

In [ ]:
Xtest, ytest = load('/content/drive/MyDrive/data/test/')
np.savez_compressed('/content/drive/MyDrive/face_data_test.npz',  Xtest, ytest)

Streaming output truncated to the last 5000 lines.
1728 (160, 160, 3)
1729 (160, 160, 3)
1730 (160, 160, 3)
1731 (160, 160, 3)
1732 (160, 160, 3)
1733 (160, 160, 3)
1734 (160, 160, 3)
1735 (160, 160, 3)
1736 (160, 160, 3)
1737 (160, 160, 3)
1738 (160, 160, 3)
1739 (160, 160, 3)
1740 (160, 160, 3)
1741 (160, 160, 3)
1742 (160, 160, 3)
1743 (160, 160, 3)
1744 (160, 160, 3)
1745 (160, 160, 3)
1746 (160, 160, 3)
1747 (160, 160, 3)
1748 (160, 160, 3)
1749 (160, 160, 3)
1750 (160, 160, 3)
1751 (160, 160, 3)
1752 (160, 160, 3)
1753 (160, 160, 3)
1754 (160, 160, 3)
1755 (160, 160, 3)
1756 (160, 160, 3)
1757 (160, 160, 3)
1758 (160, 160, 3)
1759 (160, 160, 3)
1760 (160, 160, 3)
1761 (160, 160, 3)
1762 (160, 160, 3)
1763 (160, 160, 3)
1764 (160, 160, 3)
1765 (160, 160, 3)
1766 (160, 160, 3)
1767 (160, 160, 3)
1768 (160, 160, 3)
1769 (160, 160, 3)
1770 (160, 160, 3)
1771 (160, 160, 3)
1772 (160, 160, 3)
1773 (160, 160, 3)
1774 (160, 160, 3)
1775 (160, 160, 3)
1776 (160, 160, 3)
1777 (160, 160, 3)

In [ ]:
data = np.load('/content/drive/MyDrive/face_data_test.npz')
testX, testy = data['arr_0'], data['arr_1']
print('Loaded: ', testX.shape, testy.shape)

Loaded:  (6724, 160, 160, 3) (6724,)


In [ ]:
model = load_model('/content/drive/MyDrive/facenet_keras.h5')
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [ ]:
def embed(model, f):
	f = f.astype('float32')
	m, s = f.mean(), f.std()
	f = (f - m) / s
	samples = np.expand_dims(f, axis=0)
	y = model.predict(samples)
	return y[0]

In [ ]:
nXTest = list()
for face in trainX:
	embeddings = embed(model, face)
	nXTest.append(embeddings)
nXTest = np.asarray(nXTest)
np.savez_compressed('/content/drive/MyDrive/face_test_embed.npz',nXTest, ytest)

Cause: could not parse the source code of <function <lambda> at 0x7fe63ce5f710>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7fe63ce5f710>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7fe64e4f2f80>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7fe64e4f2f80>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function <lambda> at 0x7fe64e491f80>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could no